Décomposition LU et permutations

a)
Écrivons une fonction `LU(A)` qui retourne le résultat de la décomposition LU d'une matrice.

In [34]:
import numpy as np

def LU(A):
    n=np.shape(A)[0]
    if (n!=np.shape(A)[1]):
        return "La matrice n'est pas carrée"
    U=np.copy(A)
    L=np.eye(n)
    if (U[0, 0]!=0):        
        L[1:, 0]=A[1:, 0]/U[0,0]
    else : return "La matrice n'est pas LU-décomposable"
    for i in range(1,n):
        for j in range(0, i):
            U[i,j]=0
        for j in range(i,n):
            U[i, j]= A[i, j]-np.sum(L[i, :i]*U[:i, j])
        if U[i,i]==0:
            return "La matrice n'est pas LU-décomposable"
        for j in range(i+1, n):
            L[j, i]=(A[j, i]-np.sum(L[j, :i]*U[:i, i]))/U[i, i]
    return L, U
A=np.array([[4,1,2], [2, 3, 3], [4, 7, 11]], float)
L, U = LU(A)
print(L)
print(U)
L@U

[[1.  0.  0. ]
 [0.5 1.  0. ]
 [1.  2.4 1. ]]
[[4.  1.  2. ]
 [0.  2.5 2. ]
 [0.  0.  4.2]]


array([[ 4.,  1.,  2.],
       [ 2.,  3.,  3.],
       [ 4.,  7., 11.]])

b)
Donnée la décomposition LU d'une matrice $A$, écrivons une fonction `solve(L,U,b)` qui résout le système linéaire $A\boldsymbol{x} =  \boldsymbol{b}$. 
Note : on aurait pu définir une fonction qui résoud T(x) = y avec T triangulaire supérieure, et appliquer ça à L (en transposant) puis à U, mais ici je fais les deux directement. On suppose ici implicitement (c'est a dire qu'on ne le vérifie pas) que les éléments diagonaux de U sont non nuls.

In [210]:
def solve(L, U, b):
    '''On résoud d'abord L(y)=b'''
    n= np.shape(b)[0]
    y=np.copy(b)
    for i in range(1,n):
        y[i]=b[i]-sum(y[:i]*L[i][:i])
    '''Puis on résoud U(x)= y'''
    x=np.copy(y)/U[n-1, n-1]
    for i in range(2, n+1):
        x[n-i]=(y[n-i]-sum(U[n-i, n-i+1:]*x[n-i+1:]))/U[n-i, n-i]
    return x
A=np.array([[4,1,2], [2, 3, 3], [4, 7, 11]], float)
L, U = LU(A)
x =solve(L, U, [2., 4., 5.])
print(x)
A@x

[ 0.5  2.  -1. ]


array([2., 4., 5.])

c)
Écrivons une fonction `LU_inplace(A)` qui ne crée pas de nouveaux tableaux pour $L$ et $U$ mais modifie $A$ de sorte que sa partie triangulaire inférieure (sans la diagonale) corresponde à $L$ et sa partie triangulaire supérieure (avec la diagonale) corresponde à $U$. Faisons également une version `solve_inplace` prenant en entrée la sortie de `LU_inplace` et retournant la solution $\boldsymbol{x}$, sans utiliser de nouveaux tableaux.

In [169]:
def LU_inplace(A):
    n=np.shape(A)[0]
    if (n!=np.shape(A)[1]):
        return "La matrice n'est pas carrée"
    if (A[0, 0]!=0):        
        A[1:, 0]/=A[0,0]
    for i in range(1,n):
        for j in range(i,n):
            A[i, j]-=np.sum(A[i, :i]*A[:i, j])
        if A[i,i]==0:
            return "La matrice n'est pas LU-décomposable"
        for j in range(i+1, n):
           A[j, i]=(A[j,i]-np.sum(A[j, :i]*A[:i, i]))/A[i, i]
    return A
A=np.array([[4,1,2], [2, 3, 3], [4, 7, 11]], float)
print(LU_inplace(A))

[[4.  1.  2. ]
 [0.5 2.5 2. ]
 [1.  2.4 4.2]]


On suppose dans le prochain algorithme que A est deja la sortie de LU_inplace(A):

In [211]:
def solve_inplace(A, b):
    n=np.shape(A)[0]
    for i in range(1,n):
        b[i]-=sum(b[:i]*A[i][:i])
    b[n-1]/=A[n-1, n-1]
    for i in range(2, n+1):
        b[n-i]=(b[n-i]-sum(A[n-i, n-i+1:]*b[n-i+1:]))/A[n-i, n-i]
    return b
A=np.array([[4,1,2], [2, 3, 3], [4, 7, 11]], float)
x=solve_inplace(LU_inplace(A), [2., 4., 5.])
print(A)
print(x)
np.array([[4,1,2], [2, 3, 3], [4, 7, 11]], float)@x

[[4.  1.  2. ]
 [0.5 2.5 2. ]
 [1.  2.4 4.2]]
[0.4999999999999999, 2.0, -0.9999999999999998]


array([2., 4., 5.])

d)
En utilisant la décomposition LU de la matrice $A$, écrivons une fonction `det(A)` qui retourne son déterminant.

In [212]:
def det(A):
    n=np.shape(A)[0]
    L, U = LU(A)
    P=1
    for i in range(n):
        P*=U[i, i]
    return P
A=np.array([[4,1,2], [2, 3, 3], [4, 7, 11]], float)
print(det(A))

42.0


Méthode de la puissance itérée

a)
Définissons une fonction `puissance(A, x0, k)` qui retourne $\boldsymbol{x}_k$. Avec cette fonction, déterminons le plus grand vecteur propre de la matrice:

In [236]:
def puissance(A, x0, k):
    res=x0
    for _ in range(1, k):
        res=A@res
    return res
A=np.array([[0.5, 0.5], [0.2, 0.8]])
X=puissance(A, [2, 1], 20)
v=X/np.linalg.norm(X)
print(v)

[0.70710678 0.70710678]


b)On remarque que la valeur propre associée à v est 1 : 

In [237]:
print(v)
print(A@v)

[0.70710678 0.70710678]
[0.70710678 0.70710678]


c) Écrivons un programme permettant de calculer automatiquement la valeur propre de plus grand module et le vecteur propre associé d'une matrice carrée avec une certaine précision donnée. L'idée est de comparer $Ax$ avec $\lambda x$ , en conduisant les calculs jusqu'a a voir une différence aussi petite de souhaitée :

In [20]:
def VVpropreMax(A, eps):
    n=np.shape(A)[0]
    B0=np.arange(n, dtype=float)
    lamda = 2.0
    x=B0
    while ((np.linalg.norm(A@x-lamda*x)) > eps):
        x=A@x
        x=x/np.linalg.norm(x)
        lamda = sum(A@x*x)
    return lamda, x
A=np.array([[0.5, 0.5], [0.2, 0.8]])
print(VVpropreMax(A, 0.0000000001))

(1.0000000000244076, array([0.70710678, 0.70710678]))


d)Sous Numpy, a méthode linalg.eig est la plus générale qui peut sortir toutes les valeurs propres ainsi que leurs vecteurs propres associées à une matrice : 

In [28]:
np.linalg.eig(A)

(array([0.3, 1. ]),
 array([[-0.92847669, -0.70710678],
        [ 0.37139068, -0.70710678]]))

e)Comparons la rapidité du code précédent et des fonctions Numpy pour des matrices de tailles $n\times n$ avec $n=10,100,1 000$.


In [26]:

import timeit as timeit
A=np.random.random((10,10))
print("Temps d'execution pour l'algo librairie et l'algo perso pour n=10:")
print(timeit.timeit('(np.linalg.eig(A))', number = 100, globals = globals()))
print(timeit.timeit('(VVpropreMax(A, 0.0000001))', number = 100, globals = globals()))

print("Temps d'execution pour l'algo librairie et l'algo perso pour n=100:")
A=np.random.random((100,100))
print(timeit.timeit('(np.linalg.eig(A))', number = 100, globals = globals()))
print(timeit.timeit('(VVpropreMax(A, 0.0000001))', number = 100, globals = globals()))

print("Temps d'execution pour l'algo librairie et l'algo perso pour n=1000:")
A=np.random.random((1000,1000))
print(timeit.timeit('(np.linalg.eig(A))', number = 100, globals = globals()))
print(timeit.timeit('(VVpropreMax(A, 0.0000001))', number = 100, globals = globals()))


Temps d'execution pour l'algo librairie et l'algo perso pour n=10:
0.01180050999391824
0.06805550598073751
Temps d'execution pour l'algo librairie et l'algo perso pour n=100:
1.7692958459956571
0.09946216800017282
Temps d'execution pour l'algo librairie et l'algo perso pour n=1000:
328.00187290401664
2.534424484008923


Notre algorithme est beaucoup plus efficace lorsque n est grand !En effet, il est beaucoup plus spécialisé : cela vient de la plus grande difficulté de calculer toutes les valeurs propres d'une matrice, au lieu de devoir calculer seulement la plus grande en module. Il faut en effet calculer le polynôme caractéristique et ensuite ses racines.Notons Le calcul du polynome caractéristique peut se faire assez efficacement avec la méthode de Fadeev-Le Verrier (approximativment aussi rapide que de la multiplication matricielle) et les racines avec une méthode de dichotomie en deux dimensions à la newton. Fait interessant : la multiplication matricielle est parrallélisé sur numpy (mais seulement lorsque la taille de la matrice est suffisement grande pour que cela ait un interet.)

Groupes de permutations

a)Écrivons une fonction `product(g1,g2)` qui calcule le produit de deux permutations.

In [4]:
def product(g1, g2):
    n= len(g1)
    if(n!=len(g2)):
        return print("Les permutations ne sont pas de même taille")
    return tuple(g2[g1[i]] for i in range(n))
g1= (0, 2, 3, 4, 1)
g2= (0, 2, 3, 4, 1)
product(g1, g2)

(0, 3, 4, 1, 2)

b)
Écrivons une fonction `inverse(g)` qui calcule l'inverse d'une permutation. On peut utiliser le fait que la fonction $$\begin{array}{ccccc}
\phi & : & \mathbb{N} & \to & S_n \\
 & & k & \mapsto & g^k \\
\end{array}$$
ne peut pas être injective. Il existe donc un $i$ non nul tel que $g^i=Id$

In [5]:
def inverse(g):
    Id=tuple(i for i in range(len(g)))
    g1=product(g, g)
    while(product(g1, g)!=Id):
        g1=product(g1,g)
    return g1
inverse((0, 4, 1, 2, 3))

(0, 2, 3, 4, 1)

c) 
Écrivons une fonction qui retourne la décomposition d'une permutation sous forme de cycles représentés par une liste du tuples.On va créer une fonction orbite(g, i) qui retourne sous forme d'une permuation l'orbite d'un élément de $[1, n]$ par $g$:

In [6]:
def orbite(g, i0):
    n = len(g)
    orb=[j for j in range(n)]
    i=i0
    while g[i]!=i0:
        orb[i]=g[i]
        i=g[i]
    orb[i]=i0
    return orb
print(orbite((0, 2, 3, 1, 5, 4, 6), 3))

def cycles(g):
    G=list(g[1:]).copy()
    lst=[]
    while G!=[]:
        Orb= orbite(g, G[0])
        lst.append(Orb)
        for x in Orb:
            if x!=Orb[x]:
                G.remove(x)
    C=[tuple(lst[i]) for i in range(len(lst))]
    return C
exemple = (0, 3, 6, 5, 2, 7, 4, 9, 10, 1, 8)
print(cycles(exemple))


[0, 2, 3, 1, 4, 5, 6]
[(0, 3, 2, 5, 4, 7, 6, 9, 8, 1, 10), (0, 1, 6, 3, 2, 5, 4, 7, 8, 9, 10), (0, 1, 2, 3, 4, 5, 6, 7, 10, 9, 8)]


d)Écrivons une fonction qui retourne la permutation correspondant à une liste de cycles, c'est-à-dire qui fait l'inverse de la fonction précédente.

In [7]:
def compose(L):
    res=L[0]
    for i in range(1, len(L)):
        res=product(res, L[i])
    return res
compose(cycles(exemple))

(0, 3, 6, 5, 2, 7, 4, 9, 10, 1, 8)

e)
En python un groupe $G = \langle g_1,g_2,\dots,g_k \rangle$ engendré par une famille de permutations, peut être représenté par une liste de permutations `G = [g1,g2,...,gk]`. Écrivons une fonction `orbit(G,x)` qui retourne l'orbite d'un point $x\in\{1,2,\dots,n\}$:

In [3]:
import collections

def orbit(G, x):
    orbite = {x}
    queue = collections.deque([x])
    while queue:
        y = queue.popleft()
        for g in G:
            z = g[y]
            if z not in orbite:
                orbite.add(z)
                queue.append(z)            
    return orbite
g1 = (0, 2, 3, 4, 1)
g2 = (0, 2, 1, 3, 4)
G = [g1, g2]

orbite_1 = orbit(G, 1)
print(f"L'orbite de 1 est : {orbite_1}") 

orbite_3 = orbit(G, 3)
print(f"L'orbite de 3 est : {orbite_3}")

L'orbite de 1 est : {1, 2, 3, 4}
L'orbite de 3 est : {1, 2, 3, 4}


f) Définissons une fonction `stabilizer(G,x)` qui retourne le stabilisateur d'un point $x\in\{1,2,\dots,n\}$:
$$
G_x = \{g \in G : g x = x \} \,,
$$
sous la forme d'un ensemble de générateurs.

Pour ça on utilise le théorème (ou lemme) de Schreier.

In [8]:
def stabilizer(G, x):
    orbite = {x}
    queue = collections.deque([x])
    transversal_map = {x: tuple([i for i in range(len(G[0]))])}

    while queue:
        y = queue.popleft()
        t_y = transversal_map[y]
        
        for g in G:
            z = g[y]
            
            if z not in orbite:
                orbite.add(z)
                queue.append(z)
                t_z = product(g, t_y) 
                transversal_map[z] = t_z
    stabilizer_generators = []
    
    for g in G:
        for y in orbite:
            z = g[y]
            t_y = transversal_map[y]
            t_z = transversal_map[z]
            t_z_inv = inverse(t_z)
            
            g_prime = product(g, t_y)
            
            s_g_y = product(g_prime, t_z_inv)
            
            if s_g_y[x] == x:
                stabilizer_generators.append(s_g_y)
                
    return list(set(stabilizer_generators))

stab_4_gen = stabilizer(G, 4)

print(f"\n--- f) Stabilisateur G_x ---")
print(f"Générateurs du groupe S4 : {G}")
print(f"Générateurs trouvés pour le Stabilisateur de 4 (G_4) :")
for g in stab_4_gen:
    print(f"  {g}")

all_invariant = all(g[4] == 4 for g in stab_4_gen)
print(f"Tous les générateurs laissent-ils 4 invariant ? {all_invariant}")


--- f) Stabilisateur G_x ---
Générateurs du groupe S4 : [(0, 2, 3, 4, 1), (0, 2, 1, 3, 4)]
Générateurs trouvés pour le Stabilisateur de 4 (G_4) :
  (0, 2, 1, 3, 4)
  (0, 1, 2, 3, 4)
Tous les générateurs laissent-ils 4 invariant ? True
